In [24]:
import numpy as np
import pandas as pd
import math
import copy
from random import seed
from random import random
import matplotlib.pyplot as plt  # import matplotlib for plotting and visualization
import matplotlib
import PIL
import PIL.Image
import tensorflow as tf
import pathlib
from tensorflow.keras import layers
from struct import unpack
import os

In [25]:
## images path
orig_path = pathlib.Path(r'C:\Users\istra\img\images\images')
#orig_path = pathlib.Path(r'C:\Users\istra\Google Drive\Cours HEC\Maitrise\Semestre 2\Machine_Learning\TP GROUPE\book_covers')
image_count=len(list(orig_path.glob('*/*.jpg')))
print(image_count)

0


In [14]:
orig_path.open()

PermissionError: [Errno 13] Permission denied: 'C:\\Users\\istra\\img\\images\\images'